In [1]:
%autosave 300
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir("..")
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/copilot-model-run/code/Users/Soutrik.Chowdhury/datasource_hackathon


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)
from sklearn.model_selection import train_test_split
import warnings

warnings.filterwarnings("ignore")

In [4]:
merged_data_new = pd.read_pickle("processed_data/merged_data_new_2.pkl")
weekly_data = pd.read_pickle("processed_data/weekly_data.pkl")
monthly_inventory_markers = pd.read_pickle(
    "processed_data/monthly_inventory_markers.pkl"
)
weekly_inventory_markers = pd.read_pickle("processed_data/weekly_inventory_markers.pkl")
monthly_period_inventory_markers = pd.read_pickle(
    "processed_data/monthly_period_inventory_markers.pkl"
)
df_sales_demand = pd.read_pickle("processed_data/df_sales_demand.pkl")

##### Lag features and diff of lag features for both Sales and Price

In [5]:
# Sort by 'id' and 'Date'
merged_data_new = merged_data_new.sort_values(by=["id", "Date"]).reset_index(drop=True)

In [6]:
merged_data_new.head()

,id,Client,Warehouse,Product,Date,Price,Sales,Inventory,Tag
0,0_100_10705,0,100,10705,2020-07-06,17.289999,7.0,0.0,Train
1,0_100_10705,0,100,10705,2020-07-13,17.289999,7.0,0.0,Train
2,0_100_10705,0,100,10705,2020-07-20,17.289999,7.0,0.0,Train
3,0_100_10705,0,100,10705,2020-07-27,16.570000,7.0,0.0,Train
4,0_100_10705,0,100,10705,2020-08-03,14.770000,7.0,0.0,Train


In [7]:
merged_data_new.groupby(["id", "Tag"], as_index=False).size().tail()

,id,Tag,size
30101,9_82_9829,Train,15
30102,9_82_9838,Test,13
30103,9_82_9838,Train,37
30104,9_82_9950,Test,13
30105,9_82_9950,Train,32


In [8]:
def random_noise(dataframe):
    return np.random.normal(scale=0.001, size=(len(dataframe)))

In [9]:
def lag_features(dataframe, lags, id_col, target_col):
    dataframe = dataframe.copy()
    for lag in lags:
        dataframe[f"{target_col}_lag_" + str(lag)] = dataframe.groupby([id_col])[
            target_col
        ].transform(
            lambda x: x.shift(lag)
            .fillna(method="bfill")
            .fillna(method="ffill")
            .fillna(0.0)
        ) + random_noise(
            dataframe
        )
    return dataframe

In [10]:
merged_data_new = lag_features(merged_data_new, [13, 14, 15, 16], "id", "Sales")

In [11]:
merged_data_new = lag_features(merged_data_new, [13, 14, 15, 16], "id", "Price")

In [12]:
merged_data_new.loc[merged_data_new["id"] == "0_100_10705"].tail(26)

,id,Client,Warehouse,Product,Date,Price,Sales,Inventory,Tag,Sales_lag_13,Sales_lag_14,Sales_lag_15,Sales_lag_16,Price_lag_13,Price_lag_14,Price_lag_15,Price_lag_16
157,0_100_10705,0,100,10705,2023-07-10,50.917500,4.0,0.0,Train,3.000611,0.001326,4.999410,0.999815,48.618277,0.000159,47.468014,47.471412
158,0_100_10705,0,100,10705,2023-07-17,50.916668,3.0,0.0,Train,-0.000280,2.999261,-0.000090,5.000139,-0.000520,48.620428,0.001808,47.467250
159,0_100_10705,0,100,10705,2023-07-24,50.916668,3.0,0.0,Train,2.998255,-0.002045,2.998033,-0.001829,48.621405,0.000014,48.618975,0.002114
160,0_100_10705,0,100,10705,2023-07-31,0.000000,0.0,0.0,Train,2.001395,2.999227,-0.000804,3.000094,48.284571,48.622696,0.002305,48.622006
161,0_100_10705,0,100,10705,2023-08-07,50.290000,1.0,0.0,Train,3.999689,2.000693,2.999373,-0.000085,48.279149,48.284609,48.621630,0.001840
162,0_100_10705,0,100,10705,2023-08-14,50.290000,1.0,0.0,Train,3.001459,4.000089,2.000315,3.001364,48.286212,48.279042,48.285438,48.623222
163,0_100_10705,0,100,10705,2023-08-21,50.290000,2.0,0.0,Train,5.000463,3.001258,3.999626,2.000410,48.288499,48.285664,48.279570,48.284367
164,0_100_10705,0,100,10705,2023-08-28,50.290000,1.0,0.0,Train,3.999630,5.000868,2.999888,4.000122,48.682845,48.285808,48.287734,48.281064
165,0_100_10705,0,100,10705,2023-09-04,0.000000,0.0,0.0,Train,4.000743,3.999284,4.999863,2.999868,49.486167,48.682718,48.285524,48.285759
166,0_100_10705,0,100,10705,2023-09-11,49.260000,3.0,0.0,Train,0.001479,4.000422,4.001952,5.000496,0.001452,49.486578,48.683517,48.286049


In [13]:
print(merged_data_new.isnull().sum())
print(merged_data_new.shape)

id              0
Client          0
Warehouse       0
Product         0
Date            0
Price           0
Sales           0
Inventory       0
Tag             0
Sales_lag_13    0
Sales_lag_14    0
Sales_lag_15    0
Sales_lag_16    0
Price_lag_13    0
Price_lag_14    0
Price_lag_15    0
Price_lag_16    0
dtype: int64
(1615437, 17)


##### Lag difference features

In [14]:
def calculate_lag_diff(dataframe, target_col):
    dataframe = dataframe.copy()
    diff_df = (
        dataframe.filter(regex=f"{target_col}_lag")
        .diff(axis=1)
        .dropna(axis=1, how="all")
    )
    diff_df.columns = [col + "_diff" for col in diff_df.columns]
    dataframe = pd.concat([dataframe, diff_df], axis=1)
    return dataframe

In [15]:
merged_data_new = calculate_lag_diff(merged_data_new, "Sales")
merged_data_new = calculate_lag_diff(merged_data_new, "Price")

In [16]:
merged_data_new.loc[merged_data_new["id"] == "0_100_10705"].tail(26)

,id,Client,Warehouse,Product,Date,Price,Sales,Inventory,Tag,Sales_lag_13,Sales_lag_14,Sales_lag_15,Sales_lag_16,Price_lag_13,Price_lag_14,Price_lag_15,Price_lag_16,Sales_lag_14_diff,Sales_lag_15_diff,Sales_lag_16_diff,Price_lag_14_diff,Price_lag_15_diff,Price_lag_16_diff
157,0_100_10705,0,100,10705,2023-07-10,50.917500,4.0,0.0,Train,3.000611,0.001326,4.999410,0.999815,48.618277,0.000159,47.468014,47.471412,-2.999285,4.998084,-3.999595,-48.618118,47.467856,0.003397
158,0_100_10705,0,100,10705,2023-07-17,50.916668,3.0,0.0,Train,-0.000280,2.999261,-0.000090,5.000139,-0.000520,48.620428,0.001808,47.467250,2.999540,-2.999350,5.000228,48.620948,-48.618620,47.465442
159,0_100_10705,0,100,10705,2023-07-24,50.916668,3.0,0.0,Train,2.998255,-0.002045,2.998033,-0.001829,48.621405,0.000014,48.618975,0.002114,-3.000300,3.000078,-2.999862,-48.621391,48.618961,-48.616861
160,0_100_10705,0,100,10705,2023-07-31,0.000000,0.0,0.0,Train,2.001395,2.999227,-0.000804,3.000094,48.284571,48.622696,0.002305,48.622006,0.997832,-3.000031,3.000897,0.338125,-48.620391,48.619701
161,0_100_10705,0,100,10705,2023-08-07,50.290000,1.0,0.0,Train,3.999689,2.000693,2.999373,-0.000085,48.279149,48.284609,48.621630,0.001840,-1.998996,0.998680,-2.999458,0.005460,0.337021,-48.619790
162,0_100_10705,0,100,10705,2023-08-14,50.290000,1.0,0.0,Train,3.001459,4.000089,2.000315,3.001364,48.286212,48.279042,48.285438,48.623222,0.998630,-1.999774,1.001049,-0.007170,0.006395,0.337785
163,0_100_10705,0,100,10705,2023-08-21,50.290000,2.0,0.0,Train,5.000463,3.001258,3.999626,2.000410,48.288499,48.285664,48.279570,48.284367,-1.999204,0.998368,-1.999217,-0.002835,-0.006094,0.004798
164,0_100_10705,0,100,10705,2023-08-28,50.290000,1.0,0.0,Train,3.999630,5.000868,2.999888,4.000122,48.682845,48.285808,48.287734,48.281064,1.001238,-2.000980,1.000233,-0.397037,0.001926,-0.006669
165,0_100_10705,0,100,10705,2023-09-04,0.000000,0.0,0.0,Train,4.000743,3.999284,4.999863,2.999868,49.486167,48.682718,48.285524,48.285759,-0.001459,1.000579,-1.999996,-0.803449,-0.397194,0.000235
166,0_100_10705,0,100,10705,2023-09-11,49.260000,3.0,0.0,Train,0.001479,4.000422,4.001952,5.000496,0.001452,49.486578,48.683517,48.286049,3.998943,0.001530,0.998543,49.485126,-0.803061,-0.397468


In [17]:
print(merged_data_new.isnull().sum())
print(merged_data_new.shape)

id                   0
Client               0
Warehouse            0
Product              0
Date                 0
Price                0
Sales                0
Inventory            0
Tag                  0
Sales_lag_13         0
Sales_lag_14         0
Sales_lag_15         0
Sales_lag_16         0
Price_lag_13         0
Price_lag_14         0
Price_lag_15         0
Price_lag_16         0
Sales_lag_14_diff    0
Sales_lag_15_diff    0
Sales_lag_16_diff    0
Price_lag_14_diff    0
Price_lag_15_diff    0
Price_lag_16_diff    0
dtype: int64
(1615437, 23)


##### Rolling Median and Max Min Features for both Sales and Price for different windows

In [18]:
windows = [3, 4, 5, 6]

In [19]:
def roll_stats_features(dataframe, windows, id_col, target_col, min_lag):
    dataframe = dataframe.copy()

    # Define statistics to compute
    stats = ["mean", "min", "max", "median"]

    for window in windows:
        for stat in stats:
            col_name = f"{target_col}_roll_{stat}_{window}_lag_{min_lag}"
            if stat == "mean":
                dataframe[col_name] = dataframe.groupby(id_col)[target_col].transform(
                    lambda x: x.shift(min_lag)
                    .rolling(window=window, min_periods=1)
                    .mean()
                    .fillna(method="bfill")
                    .fillna(method="ffill")
                    .fillna(0.0)
                )
            elif stat == "min":
                dataframe[col_name] = dataframe.groupby(id_col)[target_col].transform(
                    lambda x: x.shift(min_lag)
                    .rolling(window=window, min_periods=1)
                    .min()
                    .fillna(method="bfill")
                    .fillna(method="ffill")
                    .fillna(0.0)
                )
            elif stat == "max":
                dataframe[col_name] = dataframe.groupby(id_col)[target_col].transform(
                    lambda x: x.shift(min_lag)
                    .rolling(window=window, min_periods=1)
                    .max()
                    .fillna(method="bfill")
                    .fillna(method="ffill")
                    .fillna(0.0)
                )
            elif stat == "median":
                dataframe[col_name] = dataframe.groupby(id_col)[target_col].transform(
                    lambda x: x.shift(min_lag)
                    .rolling(window=window, min_periods=1)
                    .median()
                    .fillna(method="bfill")
                    .fillna(method="ffill")
                    .fillna(0.0)
                )

            # Add random noise column-wise
            dataframe[col_name] += random_noise(dataframe)

    return dataframe

In [20]:
merged_data_new = roll_stats_features(merged_data_new, windows, "id", "Sales", 13)
merged_data_new = roll_stats_features(merged_data_new, windows, "id", "Price", 13)

In [21]:
merged_data_new.loc[merged_data_new["id"] == "0_100_10705"].tail()

,id,Client,Warehouse,Product,Date,Price,Sales,Inventory,Tag,Sales_lag_13,Sales_lag_14,Sales_lag_15,Sales_lag_16,Price_lag_13,Price_lag_14,Price_lag_15,Price_lag_16,Sales_lag_14_diff,Sales_lag_15_diff,Sales_lag_16_diff,Price_lag_14_diff,Price_lag_15_diff,Price_lag_16_diff,Sales_roll_mean_3_lag_13,Sales_roll_min_3_lag_13,Sales_roll_max_3_lag_13,Sales_roll_median_3_lag_13,Sales_roll_mean_4_lag_13,Sales_roll_min_4_lag_13,Sales_roll_max_4_lag_13,Sales_roll_median_4_lag_13,Sales_roll_mean_5_lag_13,Sales_roll_min_5_lag_13,Sales_roll_max_5_lag_13,Sales_roll_median_5_lag_13,Sales_roll_mean_6_lag_13,Sales_roll_min_6_lag_13,Sales_roll_max_6_lag_13,Sales_roll_median_6_lag_13,Price_roll_mean_3_lag_13,Price_roll_min_3_lag_13,Price_roll_max_3_lag_13,Price_roll_median_3_lag_13,Price_roll_mean_4_lag_13,Price_roll_min_4_lag_13,Price_roll_max_4_lag_13,Price_roll_median_4_lag_13,Price_roll_mean_5_lag_13,Price_roll_min_5_lag_13,Price_roll_max_5_lag_13,Price_roll_median_5_lag_13,Price_roll_mean_6_lag_13,Price_roll_min_6_lag_13,Price_roll_max_6_lag_13,Price_roll_median_6_lag_13
178,0_100_10705,0,100,10705,2023-12-04,0.0,0.0,0.0,Test,-0.000440,1.000158,2.000559,0.998177,-0.001022,50.290964,50.291665,50.290009,1.000598,1.000401,-1.002382,50.291986,0.000701,-0.001656,1.000351,0.002163,1.999582,1.000010,1.000770,0.001524,1.999027,0.998843,0.998923,0.001016,2.000689,1.002053,0.834247,-0.001317,1.999851,0.999571,33.525317,0.000712,50.288385,50.287626,37.717667,0.000156,50.289679,50.290729,40.231052,0.000493,50.288590,50.289859,33.528501,0.000242,50.288906,50.290664
179,0_100_10705,0,100,10705,2023-12-11,0.0,0.0,0.0,Test,2.999671,-0.000449,0.999862,1.999576,49.259871,0.000908,50.288778,50.288883,-3.000120,1.000311,0.999714,-49.258963,50.287870,0.000105,1.332962,0.000668,3.002267,1.000543,1.499535,-0.000627,3.000673,1.500497,1.399496,0.000016,3.000455,1.001165,1.332538,-0.000551,3.000492,1.000410,33.185227,0.000502,50.289931,49.260840,37.458999,0.000141,50.288292,49.775117,40.025928,-0.000845,50.291010,50.289812,41.736170,0.001806,50.289051,50.289624
180,0_100_10705,0,100,10705,2023-12-18,0.0,0.0,0.0,Test,3.999669,3.000487,-0.000432,0.999896,49.261716,49.259371,0.000278,50.290228,-0.999182,-3.000919,1.000328,-0.002345,-49.259093,50.289950,2.334185,0.000389,4.001480,2.998946,1.999302,-0.000634,4.000261,1.999979,1.999099,0.000591,3.998862,1.998811,1.833635,0.000524,3.999229,1.498771,32.840017,-0.000445,49.262719,49.258733,37.203726,-0.000689,50.289121,49.261705,39.819117,-0.000287,50.289742,49.263386,41.564665,-0.000163,50.290150,49.776388
181,0_100_10705,0,100,10705,2023-12-25,0.0,0.0,0.0,Test,3.000594,3.999879,2.997867,-0.000840,49.261225,49.262229,49.260241,0.000741,0.999285,-1.002012,-2.998707,0.001005,-0.001988,-49.259500,3.333226,2.997137,3.999133,2.999855,2.498919,0.000040,4.001397,2.999827,2.200374,0.001187,3.999144,3.000402,2.165921,0.000725,3.999884,2.498745,49.259476,49.259767,49.262930,49.261032,36.945000,0.000642,49.262446,49.260836,39.616329,-0.000362,50.289477,49.257854,41.395257,0.001131,50.289958,49.260646
182,0_100_10705,0,100,10705,2024-01-01,0.0,0.0,0.0,Test,2.999280,3.001278,3.998447,2.999339,49.062987,49.259847,49.262020,49.259029,0.001998,0.997170,-0.999108,0.196860,0.002173,-0.002992,3.332105,2.999847,4.000153,2.999421,3.248563,3.000032,4.000397,3.001072,2.598342,-0.000918,4.001067,2.999940,2.332160,-0.002154,3.999942,3.001492,49.194670,49.062911,49.262218,49.259791,49.210717,49.064214,49.261756,49.260862,39.370567,0.000932,49.263102,49.259724,41.191477,-0.000356,50.290210,49.259831


In [22]:
print(merged_data_new.isnull().sum())
print(merged_data_new.shape)

id                            0
Client                        0
Warehouse                     0
Product                       0
Date                          0
Price                         0
Sales                         0
Inventory                     0
Tag                           0
Sales_lag_13                  0
Sales_lag_14                  0
Sales_lag_15                  0
Sales_lag_16                  0
Price_lag_13                  0
Price_lag_14                  0
Price_lag_15                  0
Price_lag_16                  0
Sales_lag_14_diff             0
Sales_lag_15_diff             0
Sales_lag_16_diff             0
Price_lag_14_diff             0
Price_lag_15_diff             0
Price_lag_16_diff             0
Sales_roll_mean_3_lag_13      0
Sales_roll_min_3_lag_13       0
Sales_roll_max_3_lag_13       0
Sales_roll_median_3_lag_13    0
Sales_roll_mean_4_lag_13      0
Sales_roll_min_4_lag_13       0
Sales_roll_max_4_lag_13       0
Sales_roll_median_4_lag_13    0
Sales_ro

##### Exponential Mean Features

In [23]:
def ewm_features(dataframe, alphas, lags, id_col, target_col):
    dataframe = dataframe.copy()
    for alpha in alphas:
        for lag in lags:
            dataframe[
                f"{target_col}_ewm_alpha_"
                + str(alpha).replace(".", "")
                + "_lag_"
                + str(lag)
            ] = dataframe.groupby([id_col])[target_col].transform(
                lambda x: x.shift(lag)
                .ewm(alpha=alpha, min_periods=1)
                .mean()
                .fillna(method="bfill")
                .fillna(method="ffill")
                .fillna(0.0)
            )
    return dataframe

In [24]:
merged_data_new = ewm_features(
    merged_data_new, [0.8, 0.85, 0.9], [13, 14], "id", "Sales"
)
merged_data_new = ewm_features(
    merged_data_new, [0.8, 0.85, 0.9], [13, 14], "id", "Price"
)

In [25]:
merged_data_new.head()

,id,Client,Warehouse,Product,Date,Price,Sales,Inventory,Tag,Sales_lag_13,Sales_lag_14,Sales_lag_15,Sales_lag_16,Price_lag_13,Price_lag_14,Price_lag_15,Price_lag_16,Sales_lag_14_diff,Sales_lag_15_diff,Sales_lag_16_diff,Price_lag_14_diff,Price_lag_15_diff,Price_lag_16_diff,Sales_roll_mean_3_lag_13,Sales_roll_min_3_lag_13,Sales_roll_max_3_lag_13,Sales_roll_median_3_lag_13,Sales_roll_mean_4_lag_13,Sales_roll_min_4_lag_13,Sales_roll_max_4_lag_13,Sales_roll_median_4_lag_13,Sales_roll_mean_5_lag_13,Sales_roll_min_5_lag_13,Sales_roll_max_5_lag_13,Sales_roll_median_5_lag_13,Sales_roll_mean_6_lag_13,Sales_roll_min_6_lag_13,Sales_roll_max_6_lag_13,Sales_roll_median_6_lag_13,Price_roll_mean_3_lag_13,Price_roll_min_3_lag_13,Price_roll_max_3_lag_13,Price_roll_median_3_lag_13,Price_roll_mean_4_lag_13,Price_roll_min_4_lag_13,Price_roll_max_4_lag_13,Price_roll_median_4_lag_13,Price_roll_mean_5_lag_13,Price_roll_min_5_lag_13,Price_roll_max_5_lag_13,Price_roll_median_5_lag_13,Price_roll_mean_6_lag_13,Price_roll_min_6_lag_13,Price_roll_max_6_lag_13,Price_roll_median_6_lag_13,Sales_ewm_alpha_08_lag_13,Sales_ewm_alpha_08_lag_14,Sales_ewm_alpha_085_lag_13,Sales_ewm_alpha_085_lag_14,Sales_ewm_alpha_09_lag_13,Sales_ewm_alpha_09_lag_14,Price_ewm_alpha_08_lag_13,Price_ewm_alpha_08_lag_14,Price_ewm_alpha_085_lag_13,Price_ewm_alpha_085_lag_14,Price_ewm_alpha_09_lag_13,Price_ewm_alpha_09_lag_14
0,0_100_10705,0,100,10705,2020-07-06,17.289999,7.0,0.0,Train,6.999956,6.999588,6.999479,7.001045,17.288617,17.287732,17.290314,17.290044,-0.000369,-0.000108,0.001566,-0.000885,0.002582,-0.000271,7.000073,7.000751,6.999460,6.999297,7.000282,6.999776,6.998860,6.999895,6.999061,6.999345,7.001079,6.999089,7.000702,7.000302,6.997820,7.001213,17.290580,17.291686,17.290472,17.290873,17.290882,17.289355,17.289072,17.288618,17.290177,17.290222,17.291164,17.289947,17.289279,17.290721,17.288134,17.289973,7.0,7.0,7.0,7.0,7.0,7.0,17.289999,17.289999,17.289999,17.289999,17.289999,17.289999
1,0_100_10705,0,100,10705,2020-07-13,17.289999,7.0,0.0,Train,7.001150,6.998959,6.999008,6.999520,17.289693,17.289902,17.289309,17.288605,-0.002191,0.000049,0.000511,0.000209,-0.000593,-0.000704,6.999984,6.998007,7.001240,7.000906,7.000889,6.999697,6.999315,6.999154,7.001336,7.000360,7.000134,6.999107,7.000980,7.000202,6.999653,6.999069,17.289972,17.291481,17.290826,17.291196,17.291098,17.288533,17.289211,17.287935,17.289454,17.290688,17.289666,17.290646,17.288963,17.289927,17.291274,17.287522,7.0,7.0,7.0,7.0,7.0,7.0,17.289999,17.289999,17.289999,17.289999,17.289999,17.289999
2,0_100_10705,0,100,10705,2020-07-20,17.289999,7.0,0.0,Train,6.996373,6.999088,7.000560,7.000849,17.289971,17.288611,17.290257,17.288971,0.002715,0.001472,0.000289,-0.001360,0.001646,-0.001287,7.000520,6.998148,7.002468,6.998770,6.999724,6.999559,6.999085,7.002249,7.000144,6.999844,7.000818,7.001587,7.000921,6.999613,6.998909,6.999116,17.289206,17.290673,17.289384,17.288859,17.289524,17.291085,17.290278,17.288658,17.289524,17.290063,17.288844,17.288623,17.290708,17.290771,17.289541,17.292272,7.0,7.0,7.0,7.0,7.0,7.0,17.289999,17.289999,17.289999,17.289999,17.289999,17.289999
3,0_100_10705,0,100,10705,2020-07-27,16.570000,7.0,0.0,Train,7.000848,6.998955,6.999397,6.999729,17.290186,17.289502,17.288695,17.290163,-0.001893,0.000442,0.000332,-0.000684,-0.000807,0.001468,7.000033,7.000362,6.999722,6.999971,6.997879,6.999637,6.999915,6.999028,6.997926,6.999233,6.998798,7.000940,7.002056,7.001288,6.999676,7.000202,17.289881,17.291029,17.288391,17.290607,17.291878,17.289465,17.288647,17.290530,17.289782,17.290300,17.290436,17.289562,17.291121,17.290790,17.287816,17.291782,7.0,7.0,7.0,7.0,7.0,7.0,17.289999,17.289999,17.289999,17.289999,17.289999,17.289999
4,0_100_10705,0,100,10705,2020-08-03,14.770000,7.0,0.0,Train,7.002479,6.999690,6.999445,7.000737,17.290153,17.289567,17.289511,17.289458,-0.002789,-0.000245,0.001292,-0.000586,-0.000057,-0.000052,7.000936,6.998244,6.998990,6.999313,7.002133,7.000203,7.000501,7.000520,6.999862,6.99996

In [26]:
print(merged_data_new.isnull().sum())
print(merged_data_new.shape)

id                            0
Client                        0
Warehouse                     0
Product                       0
Date                          0
Price                         0
Sales                         0
Inventory                     0
Tag                           0
Sales_lag_13                  0
Sales_lag_14                  0
Sales_lag_15                  0
Sales_lag_16                  0
Price_lag_13                  0
Price_lag_14                  0
Price_lag_15                  0
Price_lag_16                  0
Sales_lag_14_diff             0
Sales_lag_15_diff             0
Sales_lag_16_diff             0
Price_lag_14_diff             0
Price_lag_15_diff             0
Price_lag_16_diff             0
Sales_roll_mean_3_lag_13      0
Sales_roll_min_3_lag_13       0
Sales_roll_max_3_lag_13       0
Sales_roll_median_3_lag_13    0
Sales_roll_mean_4_lag_13      0
Sales_roll_min_4_lag_13       0
Sales_roll_max_4_lag_13       0
Sales_roll_median_4_lag_13    0
Sales_ro

##### Inventory Features

In [27]:
# Inventory Change Rate
merged_data_new["Inventory_change_rate_lag13"] = (
    merged_data_new.groupby("id")["Inventory"].shift(13).pct_change().fillna(0)
)

# Average Inventory in Last N Weeks:
for n in [4, 8, 12]:  # Adjust N as needed
    merged_data_new[f"Inventory_avg_last_{n}_weeks_lag13"] = merged_data_new.groupby(
        "id"
    )["Inventory"].transform(
        lambda x: x.shift(13)
        .rolling(window=n, min_periods=1)
        .median()
        .fillna(method="bfill")
        .fillna(method="ffill")
        .fillna(0.0)
    )

In [28]:
merged_data_new.loc[merged_data_new["id"] == "0_100_10705"].tail(13)

,id,Client,Warehouse,Product,Date,Price,Sales,Inventory,Tag,Sales_lag_13,Sales_lag_14,Sales_lag_15,Sales_lag_16,Price_lag_13,Price_lag_14,Price_lag_15,Price_lag_16,Sales_lag_14_diff,Sales_lag_15_diff,Sales_lag_16_diff,Price_lag_14_diff,Price_lag_15_diff,Price_lag_16_diff,Sales_roll_mean_3_lag_13,Sales_roll_min_3_lag_13,Sales_roll_max_3_lag_13,Sales_roll_median_3_lag_13,Sales_roll_mean_4_lag_13,Sales_roll_min_4_lag_13,Sales_roll_max_4_lag_13,Sales_roll_median_4_lag_13,Sales_roll_mean_5_lag_13,Sales_roll_min_5_lag_13,Sales_roll_max_5_lag_13,Sales_roll_median_5_lag_13,Sales_roll_mean_6_lag_13,Sales_roll_min_6_lag_13,Sales_roll_max_6_lag_13,Sales_roll_median_6_lag_13,Price_roll_mean_3_lag_13,Price_roll_min_3_lag_13,Price_roll_max_3_lag_13,Price_roll_median_3_lag_13,Price_roll_mean_4_lag_13,Price_roll_min_4_lag_13,Price_roll_max_4_lag_13,Price_roll_median_4_lag_13,Price_roll_mean_5_lag_13,Price_roll_min_5_lag_13,Price_roll_max_5_lag_13,Price_roll_median_5_lag_13,Price_roll_mean_6_lag_13,Price_roll_min_6_lag_13,Price_roll_max_6_lag_13,Price_roll_median_6_lag_13,Sales_ewm_alpha_08_lag_13,Sales_ewm_alpha_08_lag_14,Sales_ewm_alpha_085_lag_13,Sales_ewm_alpha_085_lag_14,Sales_ewm_alpha_09_lag_13,Sales_ewm_alpha_09_lag_14,Price_ewm_alpha_08_lag_13,Price_ewm_alpha_08_lag_14,Price_ewm_alpha_085_lag_13,Price_ewm_alpha_085_lag_14,Price_ewm_alpha_09_lag_13,Price_ewm_alpha_09_lag_14,Inventory_change_rate_lag13,Inventory_avg_last_4_weeks_lag13,Inventory_avg_last_8_weeks_lag13,Inventory_avg_last_12_weeks_lag13
170,0_100_10705,0,100,10705,2023-10-09,0.0,0.0,0.0,Test,4.000489,2.000904,2.000744,-0.000551,50.918389,50.919792,49.891164,0.000528,-1.999586,-0.000160,-2.001295,0.001404,-1.028629,-49.890636,2.665503,1.999164,3.999689,1.999755,1.999061,0.000661,4.000224,2.001555,1.600226,0.000234,4.000968,2.000312,1.998705,-0.001789,4.000908,2.000106,50.577741,49.890368,50.921004,50.918278,37.930464,-0.000766,50.919547,50.403972,30.344917,-0.000684,50.920298,49.889157,33.536635,-0.002754,50.920807,49.688843,3.585288,1.926440,3.693555,1.957033,3.798040,1.980401,50.493459,48.797296,50.730069,49.667962,50.858054,50.323040,0.0,0.0,0.0,0.0
171,0_100_10705,0,100,10705,2023-10-16,0.0,0.0,0.0,Test,2.998791,3.999983,1.999272,2.000630,50.917768,50.918141,50.921192,49.890799,1.001192,-2.000712,0.001359,0.000373,0.003051,-1.030392,2.998049,1.999042,3.997834,2.999861,2.749110,1.998922,4.001240,2.499322,2.199088,-0.001231,3.998305,2.001661,1.834131,0.001114,3.999169,1.998738,50.916362,50.915750,50.920266,50.917668,50.661325,49.890886,50.921727,50.916600,40.527140,0.000626,50.920631,50.916669,33.773378,-0.001159,50.919659,50.405418,3.117058,3.585288,3.104033,3.693555,3.079804,3.798040,50.832026,50.493459,50.888678,50.730069,50.910807,50.858054,0.0,0.0,0.0,0.0
172,0_100_10705,0,100,10705,2023-10-23,0.0,0.0,0.0,Test,3.000545,2.999040,3.998478,1.997864,50.916698,50.916031,50.917749,50.918146,-0.001505,0.999437,-2.000613,-0.000667,0.001718,0.000397,3.332118,2.998793,3.999151,3.000725,3.000677,1.999112,4.000619,2.998636,2.799160,2.001649,3.998557,3.000676,2.332807,-0.000928,3.999371,2.499130,50.917419,50.917520,50.917098,50.915619,50.917785,50.917575,50.920743,50.919262,50.711374,49.890001,50.920056,50.916243,42.259235,-0.000576,50.920971,50.915621,3.023412,3.117058,3.015605,3.104033,3.007980,3.079804,50.899740,50.832026,50.912470,50.888678,50.916082,50.910807,0.0,0.0,0.0,0.0
173,0_100_10705,0,100,10705,2023-10-30,0.0,0.0,0.0,Test,-0.000792,2.999780,3.000055,3.999483,-0.001407,50.916962,50.917365,50.918320,3.000572,0.000275,0.999428,50.918369,0.000403,0.000956,2.000512,0.000773,3.001021,3.000366,2.500054,0.000751,4.000306,2.999233,2.399244,0.000105,4.001368,2.999844,2.333201,0.000367,4.000377,2.498975,33.944085,-0.000097,50.915798,50.917074,38.189304,-0.001323,50.918049,50.916165,40.733757,-0.000200,50.917763,50.916672,42.260692,-0.000068,50.920609,50.917904,0.604682,3.023412,0.452341,3.015605,0.300798,3.007980,10.179948,50.899740,7.636870,50.912470,5.091608,50.916082,0.0,0.0

In [29]:
print(merged_data_new.isnull().sum())
print(merged_data_new.shape)

id                                   0
Client                               0
Warehouse                            0
Product                              0
Date                                 0
Price                                0
Sales                                0
Inventory                            0
Tag                                  0
Sales_lag_13                         0
Sales_lag_14                         0
Sales_lag_15                         0
Sales_lag_16                         0
Price_lag_13                         0
Price_lag_14                         0
Price_lag_15                         0
Price_lag_16                         0
Sales_lag_14_diff                    0
Sales_lag_15_diff                    0
Sales_lag_16_diff                    0
Price_lag_14_diff                    0
Price_lag_15_diff                    0
Price_lag_16_diff                    0
Sales_roll_mean_3_lag_13             0
Sales_roll_min_3_lag_13              0
Sales_roll_max_3_lag_13  

In [30]:
# Save the data
os.makedirs("engineered_data", exist_ok=True)
merged_data_new.to_pickle("engineered_data/merged_data_target_engineered_weekly.pkl")

##### calendar features

In [31]:
weekly_data.head()

,Date,marker_key,Year,Month,WeekOfYear,DayOfMonth,Quarter,DayOfWeek,IsWeekend,Month_sin,Month_cos,WeekOfYear_sin,WeekOfYear_cos,month_period
0,2020-07-06,1,2020,7,28,6,3,0,0,-0.500000,-0.866025,-0.239316,-0.970942,Month_Start
1,2020-07-13,1,2020,7,29,13,3,0,0,-0.500000,-0.866025,-0.354605,-0.935016,Month_Middle
2,2020-07-20,1,2020,7,30,20,3,0,0,-0.500000,-0.866025,-0.464723,-0.885456,Month_Middle
3,2020-07-27,1,2020,7,31,27,3,0,0,-0.500000,-0.866025,-0.568065,-0.822984,Month_End
4,2020-08-03,1,2020,8,32,3,3,0,0,-0.866025,-0.500000,-0.663123,-0.748511,Month_Start


In [32]:
merged_data_new = pd.merge(
    merged_data_new,
    weekly_data[
        [
            "Date",
            "Month",
            "WeekOfYear",
            "DayOfMonth",
            "Quarter",
            "Month_sin",
            "Month_cos",
            "WeekOfYear_sin",
            "WeekOfYear_cos",
            "month_period",
        ]
    ],
    on=["Date"],
    how="left",
)

In [33]:
print(merged_data_new.isnull().sum())
print(merged_data_new.shape)

id                                   0
Client                               0
Warehouse                            0
Product                              0
Date                                 0
Price                                0
Sales                                0
Inventory                            0
Tag                                  0
Sales_lag_13                         0
Sales_lag_14                         0
Sales_lag_15                         0
Sales_lag_16                         0
Price_lag_13                         0
Price_lag_14                         0
Price_lag_15                         0
Price_lag_16                         0
Sales_lag_14_diff                    0
Sales_lag_15_diff                    0
Sales_lag_16_diff                    0
Price_lag_14_diff                    0
Price_lag_15_diff                    0
Price_lag_16_diff                    0
Sales_roll_mean_3_lag_13             0
Sales_roll_min_3_lag_13              0
Sales_roll_max_3_lag_13  

##### markers like new combinations and discontinuation of products

In [34]:
# Calculate the number of weeks each ID has data
weeks_per_id = (
    merged_data_new.loc[
        (merged_data_new["Tag"] == "Train") & (merged_data_new["Sales"] > 0)
    ]
    .groupby("id")["Date"]
    .nunique()
)

# Filter IDs with less than 26 weeks of data
ids_less_than_6_months = weeks_per_id[weeks_per_id < 26].index.tolist()

In [35]:
len(ids_less_than_6_months)

7277

In [36]:
merged_data_new[merged_data_new["id"].isin(ids_less_than_6_months[:1])]

,id,Client,Warehouse,Product,Date,Price,Sales,Inventory,Tag,Sales_lag_13,Sales_lag_14,Sales_lag_15,Sales_lag_16,Price_lag_13,Price_lag_14,Price_lag_15,Price_lag_16,Sales_lag_14_diff,Sales_lag_15_diff,Sales_lag_16_diff,Price_lag_14_diff,Price_lag_15_diff,Price_lag_16_diff,Sales_roll_mean_3_lag_13,Sales_roll_min_3_lag_13,Sales_roll_max_3_lag_13,Sales_roll_median_3_lag_13,Sales_roll_mean_4_lag_13,Sales_roll_min_4_lag_13,Sales_roll_max_4_lag_13,Sales_roll_median_4_lag_13,Sales_roll_mean_5_lag_13,Sales_roll_min_5_lag_13,Sales_roll_max_5_lag_13,Sales_roll_median_5_lag_13,Sales_roll_mean_6_lag_13,Sales_roll_min_6_lag_13,Sales_roll_max_6_lag_13,Sales_roll_median_6_lag_13,Price_roll_mean_3_lag_13,Price_roll_min_3_lag_13,Price_roll_max_3_lag_13,Price_roll_median_3_lag_13,Price_roll_mean_4_lag_13,Price_roll_min_4_lag_13,Price_roll_max_4_lag_13,Price_roll_median_4_lag_13,Price_roll_mean_5_lag_13,Price_roll_min_5_lag_13,Price_roll_max_5_lag_13,Price_roll_median_5_lag_13,Price_roll_mean_6_lag_13,Price_roll_min_6_lag_13,Price_roll_max_6_lag_13,Price_roll_median_6_lag_13,Sales_ewm_alpha_08_lag_13,Sales_ewm_alpha_08_lag_14,Sales_ewm_alpha_085_lag_13,Sales_ewm_alpha_085_lag_14,Sales_ewm_alpha_09_lag_13,Sales_ewm_alpha_09_lag_14,Price_ewm_alpha_08_lag_13,Price_ewm_alpha_08_lag_14,Price_ewm_alpha_085_lag_13,Price_ewm_alpha_085_lag_14,Price_ewm_alpha_09_lag_13,Price_ewm_alpha_09_lag_14,Inventory_change_rate_lag13,Inventory_avg_last_4_weeks_lag13,Inventory_avg_last_8_weeks_lag13,Inventory_avg_last_12_weeks_lag13,Month,WeekOfYear,DayOfMonth,Quarter,Month_sin,Month_cos,WeekOfYear_sin,WeekOfYear_cos,month_period
6851,0_120_11552,0,120,11552,2023-09-04,0.000000,15.0,0.0,Train,14.999725,15.001086,14.999843,15.000591,0.000684,0.000260,0.000394,0.001203,0.001360,-0.001243,0.000748,-0.000424,0.000134,0.000809,14.999058,15.000942,14.999698,14.998609,14.999310,15.000387,14.998276,14.999591,15.000893,15.002073,15.001934,14.999329,14.999847,14.999966,15.000422,15.000244,0.001897,-0.000396,0.000385,-0.001503,-0.001004,0.000021,0.000389,-0.000750,0.000872,-0.001543,0.000641,0.000201,0.000771,0.000754,-0.000161,0.000567,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,9,36,4,3,-1.000000e+00,-1.836970e-16,-0.935016,-0.354605,Month_Start
6852,0_120_11552,0,120,11552,2023-09-11,0.000000,0.0,0.0,Train,14.999149,15.001420,15.000518,15.000082,-0.000261,-0.000500,0.001104,0.000616,0.002271,-0.000902,-0.000437,-0.000239,0.001604,-0.000488,15.000439,14.998457,15.000567,15.001208,15.001096,15.000717,15.001524,14.999690,14.998945,15.002061,15.001103,15.000303,15.002186,14.998154,14.999543,14.998694,-0.000388,0.000035,0.001449,-0.000382,-0.000145,0.000352,-0.002430,-0.000503,0.000477,0.000880,-0.000823,0.000480,0.000165,-0.000846,-0.001440,-0.000774,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,9,37,11,3,-1.000000e+00,-1.836970e-16,-0.970942,-0.239316,Month_Middle
6853,0_120_11552,0,120,11552,2023-09-18,0.000000,0.0,0.0,Train,15.001289,14.999720,15.001864,15.000305,-0.001028,-0.001953,0.000443,-0.000653,-0.001570,0.002144,-0.001558,-0.000926,0.002397,-0.001096,15.000313,14.999919,14.999599,14.998923,15.001320,14.998002,14.999772,15.000173,15.000015,14.998785,14.998530,14.999042,15.001487,14.997876,15.001350,14.999668,0.000096,0.000876,-0.000746,0.000834,0.000148,-0.000146,-0.000257,0.000246,0.000419,0.001476,0.000396,-0.000701,0.002652,0.001060,0.001215,0.002110,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,9,38,18,3,-1.000000e+00,-1.836970e-16,-0.992709,-0.120537,Month_Middle
6854,0_120_11552,0,120,11552,2023-09-25,29.720001,10.0,0.0,Train,14.999192,15.001980,14.999752,15.000666,-0.001620,-0.000008,0.000059,-0.001602,0.002788,-0.002228,0.000914,0.001612,0.000067,-0.001662,14.999959,15.000235,15.000309,15.000413,14.998361,15.00008

In [37]:
# Define the end date and start date for the last 12 months
end_date = merged_data_new.loc[merged_data_new["Tag"] == "Train"]["Date"].max()
print("End Date:", end_date)
back_date = end_date - pd.DateOffset(months=12)
print("Back Date:", back_date)

End Date: 2023-10-02 00:00:00
Back Date: 2022-10-02 00:00:00


In [38]:
# Filter training data within the last 12 months
recent_data = merged_data_new[
    (merged_data_new["Tag"] == "Train") & (merged_data_new["Date"] >= back_date)
]

# Find IDs with zero sales in the last 12 months
zero_sales_ids = (
    recent_data.groupby("id")
    .apply(lambda df: (df["Sales"] == 0).all())
    .reset_index(name="has_zero_sales")
    .query("has_zero_sales == True")["id"]
    .tolist()
)

In [39]:
len(zero_sales_ids)

1465

In [40]:
# merged_data_new[merged_data_new["id"].isin(zero_sales_ids[:1])].tail(20)

In [41]:
# adding markers for new combinations
merged_data_new["new_combination"] = np.where(
    merged_data_new["id"].isin(ids_less_than_6_months), 1, 0
)

merged_data_new["last_N_zero_sales"] = np.where(
    merged_data_new["id"].isin(zero_sales_ids), 1, 0
)

In [42]:
merged_data_new.head()

,id,Client,Warehouse,Product,Date,Price,Sales,Inventory,Tag,Sales_lag_13,Sales_lag_14,Sales_lag_15,Sales_lag_16,Price_lag_13,Price_lag_14,Price_lag_15,Price_lag_16,Sales_lag_14_diff,Sales_lag_15_diff,Sales_lag_16_diff,Price_lag_14_diff,Price_lag_15_diff,Price_lag_16_diff,Sales_roll_mean_3_lag_13,Sales_roll_min_3_lag_13,Sales_roll_max_3_lag_13,Sales_roll_median_3_lag_13,Sales_roll_mean_4_lag_13,Sales_roll_min_4_lag_13,Sales_roll_max_4_lag_13,Sales_roll_median_4_lag_13,Sales_roll_mean_5_lag_13,Sales_roll_min_5_lag_13,Sales_roll_max_5_lag_13,Sales_roll_median_5_lag_13,Sales_roll_mean_6_lag_13,Sales_roll_min_6_lag_13,Sales_roll_max_6_lag_13,Sales_roll_median_6_lag_13,Price_roll_mean_3_lag_13,Price_roll_min_3_lag_13,Price_roll_max_3_lag_13,Price_roll_median_3_lag_13,Price_roll_mean_4_lag_13,Price_roll_min_4_lag_13,Price_roll_max_4_lag_13,Price_roll_median_4_lag_13,Price_roll_mean_5_lag_13,Price_roll_min_5_lag_13,Price_roll_max_5_lag_13,Price_roll_median_5_lag_13,Price_roll_mean_6_lag_13,Price_roll_min_6_lag_13,Price_roll_max_6_lag_13,Price_roll_median_6_lag_13,Sales_ewm_alpha_08_lag_13,Sales_ewm_alpha_08_lag_14,Sales_ewm_alpha_085_lag_13,Sales_ewm_alpha_085_lag_14,Sales_ewm_alpha_09_lag_13,Sales_ewm_alpha_09_lag_14,Price_ewm_alpha_08_lag_13,Price_ewm_alpha_08_lag_14,Price_ewm_alpha_085_lag_13,Price_ewm_alpha_085_lag_14,Price_ewm_alpha_09_lag_13,Price_ewm_alpha_09_lag_14,Inventory_change_rate_lag13,Inventory_avg_last_4_weeks_lag13,Inventory_avg_last_8_weeks_lag13,Inventory_avg_last_12_weeks_lag13,Month,WeekOfYear,DayOfMonth,Quarter,Month_sin,Month_cos,WeekOfYear_sin,WeekOfYear_cos,month_period,new_combination,last_N_zero_sales
0,0_100_10705,0,100,10705,2020-07-06,17.289999,7.0,0.0,Train,6.999956,6.999588,6.999479,7.001045,17.288617,17.287732,17.290314,17.290044,-0.000369,-0.000108,0.001566,-0.000885,0.002582,-0.000271,7.000073,7.000751,6.999460,6.999297,7.000282,6.999776,6.998860,6.999895,6.999061,6.999345,7.001079,6.999089,7.000702,7.000302,6.997820,7.001213,17.290580,17.291686,17.290472,17.290873,17.290882,17.289355,17.289072,17.288618,17.290177,17.290222,17.291164,17.289947,17.289279,17.290721,17.288134,17.289973,7.0,7.0,7.0,7.0,7.0,7.0,17.289999,17.289999,17.289999,17.289999,17.289999,17.289999,0.0,0.0,0.0,0.0,7,28,6,3,-0.500000,-0.866025,-0.239316,-0.970942,Month_Start,0,0
1,0_100_10705,0,100,10705,2020-07-13,17.289999,7.0,0.0,Train,7.001150,6.998959,6.999008,6.999520,17.289693,17.289902,17.289309,17.288605,-0.002191,0.000049,0.000511,0.000209,-0.000593,-0.000704,6.999984,6.998007,7.001240,7.000906,7.000889,6.999697,6.999315,6.999154,7.001336,7.000360,7.000134,6.999107,7.000980,7.000202,6.999653,6.999069,17.289972,17.291481,17.290826,17.291196,17.291098,17.288533,17.289211,17.287935,17.289454,17.290688,17.289666,17.290646,17.288963,17.289927,17.291274,17.287522,7.0,7.0,7.0,7.0,7.0,7.0,17.289999,17.289999,17.289999,17.289999,17.289999,17.289999,0.0,0.0,0.0,0.0,7,29,13,3,-0.500000,-0.866025,-0.354605,-0.935016,Month_Middle,0,0
2,0_100_10705,0,100,10705,2020-07-20,17.289999,7.0,0.0,Train,6.996373,6.999088,7.000560,7.000849,17.289971,17.288611,17.290257,17.288971,0.002715,0.001472,0.000289,-0.001360,0.001646,-0.001287,7.000520,6.998148,7.002468,6.998770,6.999724,6.999559,6.999085,7.002249,7.000144,6.999844,7.000818,7.001587,7.000921,6.999613,6.998909,6.999116,17.289206,17.290673,17.289384,17.288859,17.289524,17.291085,17.290278,17.288658,17.289524,17.290063,17.288844,17.288623,17.290708,17.290771,17.289541,17.292272,7.0,7.0,7.0,7.0,7.0,7.0,17.289999,17.289999,17.289999,17.289999,17.289999,17.289999,0.0,0.0,0.0,0.0,7,30,20,3,-0.500000,-0.866025,-0.464723,-0.885456,Month_Middle,0,0
3,0_100_10705,0,100,10705,2020-07-27,16.570000,7.0,0.0,Train,7.000848,6.998955,6.999397,6.999729,17.290186,17.289502,17.288695,17.290163,-0.001893,0.000442,0.000332,-0.000684,-0.000807,0.001468,7.000033,7.000362,6.999722,6.999971,6.997879,6.999637,6.999915,6.999028,6.997926,6.999233,6.998798,7.000940,7.002056,7.001288,6.999676,7.000202,17.289881,17.291029,

In [43]:
print(merged_data_new.isnull().sum())
print(merged_data_new.shape)

id                                   0
Client                               0
Warehouse                            0
Product                              0
Date                                 0
Price                                0
Sales                                0
Inventory                            0
Tag                                  0
Sales_lag_13                         0
Sales_lag_14                         0
Sales_lag_15                         0
Sales_lag_16                         0
Price_lag_13                         0
Price_lag_14                         0
Price_lag_15                         0
Price_lag_16                         0
Sales_lag_14_diff                    0
Sales_lag_15_diff                    0
Sales_lag_16_diff                    0
Price_lag_14_diff                    0
Price_lag_15_diff                    0
Price_lag_16_diff                    0
Sales_roll_mean_3_lag_13             0
Sales_roll_min_3_lag_13              0
Sales_roll_max_3_lag_13  

##### adding additional features

In [51]:
monthly_inventory_markers.head()

,id,Month,monthly_inventory_category
0,0_100_10705,1,Low
1,0_100_10705,2,Low
2,0_100_10705,3,Low
3,0_100_10705,4,Low
4,0_100_10705,5,Low


In [48]:
weekly_inventory_markers.head()

,id,WeekOfYear,weekly_inventory_category
0,0_100_10705,1,Low
1,0_100_10705,2,Low
2,0_100_10705,3,Low
3,0_100_10705,4,Low
4,0_100_10705,5,Low


In [49]:
monthly_period_inventory_markers.head()

,id,month_period,monthly_period_inventory_category
0,0_100_10705,Month_End,Low
1,0_100_10705,Month_Middle,Low
2,0_100_10705,Month_Start,Low
3,0_100_10789,Month_End,Low
4,0_100_10789,Month_Middle,Low


In [50]:
df_sales_demand.head()

,id,Sales,demand_category
0,0_100_10705,"[7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 1.0, 0.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 3.0, 0.0, 5.0, 7.0, 4.0, 0.0, 3.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 5.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, ...]",Irregular
1,0_100_10789,"[14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 20.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 20.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 2.0, 0.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 6.0, 0.0, 10.0, 14.0, 8.0, 0.0, 6.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 10.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 20.0, 21.0, 21.0, 21.0, 15.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, ...]",Regular
2,0_100_11015,"[7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 1.0, 0.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 3.0, 0.0, 5.0, 7.0, 4.0, 0.0, 3.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 5.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, ...]",Irregular
3,0_100_1149,"[7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 0.0, 0.0, 0.0, 0.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 3.0, 0.0, 5.0, 7.0, 4.0, 0.0, 3.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 5.0, 7.0, 7.0, 7.0, 7.0, 0.0, 0.0, 0.0, 0.0, 5.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, ...]",Irregular
4,0_100_11526,"[2.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 1.0, 0.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 3.0, 0.0, 5.0, 7.0, 4.0, 0.0, 3.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 5.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, ...]",Irregular


In [46]:
# join monthly inventory markers
merged_data_new_extras = pd.merge(
    merged_data_new,
    monthly_inventory_markers,
    on=["id", "Month"],
    how="left",
)

# join weekly inventory markers
merged_data_new_extras = pd.merge(
    merged_data_new_extras,
    weekly_inventory_markers,
    on=["id", "WeekOfYear"],
    how="left",
)

# join monthly period inventory markers
merged_data_new_extras = pd.merge(
    merged_data_new_extras,
    monthly_period_inventory_markers,
    on=["id", "month_period"],
    how="left",
)

# join sales demand data
merged_data_new_extras = pd.merge(
    merged_data_new_extras,
    df_sales_demand[["id", "demand_category"]],
    on=["id"],
    how="left",
)

In [52]:
merged_data_new_extras.head()

,id,Client,Warehouse,Product,Date,Price,Sales,Inventory,Tag,Sales_lag_13,Sales_lag_14,Sales_lag_15,Sales_lag_16,Price_lag_13,Price_lag_14,Price_lag_15,Price_lag_16,Sales_lag_14_diff,Sales_lag_15_diff,Sales_lag_16_diff,Price_lag_14_diff,Price_lag_15_diff,Price_lag_16_diff,Sales_roll_mean_3_lag_13,Sales_roll_min_3_lag_13,Sales_roll_max_3_lag_13,Sales_roll_median_3_lag_13,Sales_roll_mean_4_lag_13,Sales_roll_min_4_lag_13,Sales_roll_max_4_lag_13,Sales_roll_median_4_lag_13,Sales_roll_mean_5_lag_13,Sales_roll_min_5_lag_13,Sales_roll_max_5_lag_13,Sales_roll_median_5_lag_13,Sales_roll_mean_6_lag_13,Sales_roll_min_6_lag_13,Sales_roll_max_6_lag_13,Sales_roll_median_6_lag_13,Price_roll_mean_3_lag_13,Price_roll_min_3_lag_13,Price_roll_max_3_lag_13,Price_roll_median_3_lag_13,Price_roll_mean_4_lag_13,Price_roll_min_4_lag_13,Price_roll_max_4_lag_13,Price_roll_median_4_lag_13,Price_roll_mean_5_lag_13,Price_roll_min_5_lag_13,Price_roll_max_5_lag_13,Price_roll_median_5_lag_13,Price_roll_mean_6_lag_13,Price_roll_min_6_lag_13,Price_roll_max_6_lag_13,Price_roll_median_6_lag_13,Sales_ewm_alpha_08_lag_13,Sales_ewm_alpha_08_lag_14,Sales_ewm_alpha_085_lag_13,Sales_ewm_alpha_085_lag_14,Sales_ewm_alpha_09_lag_13,Sales_ewm_alpha_09_lag_14,Price_ewm_alpha_08_lag_13,Price_ewm_alpha_08_lag_14,Price_ewm_alpha_085_lag_13,Price_ewm_alpha_085_lag_14,Price_ewm_alpha_09_lag_13,Price_ewm_alpha_09_lag_14,Inventory_change_rate_lag13,Inventory_avg_last_4_weeks_lag13,Inventory_avg_last_8_weeks_lag13,Inventory_avg_last_12_weeks_lag13,Month,WeekOfYear,DayOfMonth,Quarter,Month_sin,Month_cos,WeekOfYear_sin,WeekOfYear_cos,month_period,new_combination,last_N_zero_sales,monthly_inventory_category,weekly_inventory_category,monthly_period_inventory_category,demand_category
0,0_100_10705,0,100,10705,2020-07-06,17.289999,7.0,0.0,Train,6.999956,6.999588,6.999479,7.001045,17.288617,17.287732,17.290314,17.290044,-0.000369,-0.000108,0.001566,-0.000885,0.002582,-0.000271,7.000073,7.000751,6.999460,6.999297,7.000282,6.999776,6.998860,6.999895,6.999061,6.999345,7.001079,6.999089,7.000702,7.000302,6.997820,7.001213,17.290580,17.291686,17.290472,17.290873,17.290882,17.289355,17.289072,17.288618,17.290177,17.290222,17.291164,17.289947,17.289279,17.290721,17.288134,17.289973,7.0,7.0,7.0,7.0,7.0,7.0,17.289999,17.289999,17.289999,17.289999,17.289999,17.289999,0.0,0.0,0.0,0.0,7,28,6,3,-0.500000,-0.866025,-0.239316,-0.970942,Month_Start,0,0,Low,Low,Low,Irregular
1,0_100_10705,0,100,10705,2020-07-13,17.289999,7.0,0.0,Train,7.001150,6.998959,6.999008,6.999520,17.289693,17.289902,17.289309,17.288605,-0.002191,0.000049,0.000511,0.000209,-0.000593,-0.000704,6.999984,6.998007,7.001240,7.000906,7.000889,6.999697,6.999315,6.999154,7.001336,7.000360,7.000134,6.999107,7.000980,7.000202,6.999653,6.999069,17.289972,17.291481,17.290826,17.291196,17.291098,17.288533,17.289211,17.287935,17.289454,17.290688,17.289666,17.290646,17.288963,17.289927,17.291274,17.287522,7.0,7.0,7.0,7.0,7.0,7.0,17.289999,17.289999,17.289999,17.289999,17.289999,17.289999,0.0,0.0,0.0,0.0,7,29,13,3,-0.500000,-0.866025,-0.354605,-0.935016,Month_Middle,0,0,Low,Low,Low,Irregular
2,0_100_10705,0,100,10705,2020-07-20,17.289999,7.0,0.0,Train,6.996373,6.999088,7.000560,7.000849,17.289971,17.288611,17.290257,17.288971,0.002715,0.001472,0.000289,-0.001360,0.001646,-0.001287,7.000520,6.998148,7.002468,6.998770,6.999724,6.999559,6.999085,7.002249,7.000144,6.999844,7.000818,7.001587,7.000921,6.999613,6.998909,6.999116,17.289206,17.290673,17.289384,17.288859,17.289524,17.291085,17.290278,17.288658,17.289524,17.290063,17.288844,17.288623,17.290708,17.290771,17.289541,17.292272,7.0,7.0,7.0,7.0,7.0,7.0,17.289999,17.289999,17.289999,17.289999,17.289999,17.289999,0.0,0.0,0.0,0.0,7,30,20,3,-0.500000,-0.866025,-0.464723,-0.885456,Month_Middle,0,0,Low,Low,Low,Irregular
3,0_100_10705,0,100,10705,2020-07-27,16.570000,7.0,0.0,Train,7.000848,6.998955,6.999397,6.999729,17.290186,17.289502,17.288695,17.290163,-0.001893,0.000442,0.000332,-0.000684,-0.000807,0.00

In [53]:
print(merged_data_new_extras.isnull().sum())
print(merged_data_new_extras.shape)

id                                       0
Client                                   0
Warehouse                                0
Product                                  0
Date                                     0
Price                                    0
Sales                                    0
Inventory                                0
Tag                                      0
Sales_lag_13                             0
Sales_lag_14                             0
Sales_lag_15                             0
Sales_lag_16                             0
Price_lag_13                             0
Price_lag_14                             0
Price_lag_15                             0
Price_lag_16                             0
Sales_lag_14_diff                        0
Sales_lag_15_diff                        0
Sales_lag_16_diff                        0
Price_lag_14_diff                        0
Price_lag_15_diff                        0
Price_lag_16_diff                        0
Sales_roll_

In [54]:
merged_data_new_extras[['monthly_inventory_category']] = merged_data_new_extras[['monthly_inventory_category']].fillna('Unknown')
merged_data_new_extras[['weekly_inventory_category']] = merged_data_new_extras[['weekly_inventory_category']].fillna('Unknown')
merged_data_new_extras[['monthly_period_inventory_category']] = merged_data_new_extras[['monthly_period_inventory_category']].fillna('Unknown')

In [55]:
print(merged_data_new_extras.isnull().sum())
print(merged_data_new_extras.shape)

id                                   0
Client                               0
Warehouse                            0
Product                              0
Date                                 0
Price                                0
Sales                                0
Inventory                            0
Tag                                  0
Sales_lag_13                         0
Sales_lag_14                         0
Sales_lag_15                         0
Sales_lag_16                         0
Price_lag_13                         0
Price_lag_14                         0
Price_lag_15                         0
Price_lag_16                         0
Sales_lag_14_diff                    0
Sales_lag_15_diff                    0
Sales_lag_16_diff                    0
Price_lag_14_diff                    0
Price_lag_15_diff                    0
Price_lag_16_diff                    0
Sales_roll_mean_3_lag_13             0
Sales_roll_min_3_lag_13              0
Sales_roll_max_3_lag_13  

In [ ]:
merged_data_new_extras.to_pickle(
    "engineered_data/merged_data_engineered_all_features_weekly.pkl"
)

################################################# end of script ##########################################################